##  Question 3

In [19]:
# import required libraries
from bs4 import BeautifulSoup
import os, re
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

###  Bigram Inverted Index

In [20]:
# create empty inverted index 
bigram_inv_index = {}


# set path to access 1400 files
path = "G:\\IIITD\\Sem 2\\IR\\assignment\\A1\\dataset_preprocessed"
os.chdir(path)


# iterate over all files  
for doc_id, file in enumerate(os.listdir()):
    curr_file = open(file)
    words = curr_file.read().split()
        
    # iterate over all bigrams and add to inverted index
    for i in range(len(words)-1):
        bigram = words[i] + " " + words[i+1]
        if bigram not in bigram_inv_index:
            bigram_inv_index[bigram] = {"doc_freq" : 1, "docs": [file]}
        else:
            if file not in bigram_inv_index[bigram]["docs"]:
                bigram_inv_index[bigram]["docs"].append(file)
                bigram_inv_index[bigram]["doc_freq"] += 1
                                        
        
print("total bigram :", len(bigram_inv_index.keys()))

total bigram : 85198


In [21]:
bigram_inv_index

{'experimental investigation': {'doc_freq': 50,
  'docs': ['cranfield0001',
   'cranfield0029',
   'cranfield0084',
   'cranfield0170',
   'cranfield0173',
   'cranfield0176',
   'cranfield0189',
   'cranfield0245',
   'cranfield0251',
   'cranfield0372',
   'cranfield0423',
   'cranfield0442',
   'cranfield0497',
   'cranfield0505',
   'cranfield0522',
   'cranfield0552',
   'cranfield0569',
   'cranfield0635',
   'cranfield0636',
   'cranfield0662',
   'cranfield0694',
   'cranfield0713',
   'cranfield0766',
   'cranfield0772',
   'cranfield0801',
   'cranfield0816',
   'cranfield0836',
   'cranfield0844',
   'cranfield0858',
   'cranfield0887',
   'cranfield0932',
   'cranfield0986',
   'cranfield0996',
   'cranfield1019',
   'cranfield1039',
   'cranfield1074',
   'cranfield1092',
   'cranfield1097',
   'cranfield1098',
   'cranfield1156',
   'cranfield1159',
   'cranfield1161',
   'cranfield1205',
   'cranfield1220',
   'cranfield1225',
   'cranfield1227',
   'cranfield1230',
   '

#### Pickle the bigram inverted index

In [22]:
# pickle the bigram inverted index
path = "G:\\IIITD\\Sem 2\\IR\\assignment\\A1"
os.chdir(path)


with open("bigram_inverted_index.pickle", "wb") as f:
    pickle.dump(bigram_inv_index, f)
    print("Inveted index pickled sucessfully")

Inveted index pickled sucessfully


In [23]:
# load the bigram inverted index
with open("bigram_inverted_index.pickle", "rb") as f:
    bigram_inv_index = pickle.load(f)
    print("Inverted index loaded sucessfully")

Inverted index loaded sucessfully


### Positional index

In [25]:
# set path where files are present
path = "G:\\IIITD\\Sem 2\\IR\\assignment\\A1\\dataset_preprocessed"
os.chdir(path)


# initalize empty positional index
positional_index = {}


# iterate over each file and parse terms and its position
for file_num, file in enumerate(os.listdir()):
    with open(file, 'r') as f:
        all_words = f.read().split()
        unique_words = set(all_words) 
        f.close()
    
    # add postion of each unique terms into postional index
    for unique_word in unique_words:
        for word_pos, word in enumerate(all_words):
            if unique_word == word:
                if unique_word not in positional_index:
                    positional_index[unique_word] = {"doc_freq": 0, "docs": {}}
                    positional_index[unique_word]["docs"][file] = [word_pos]
                    positional_index[unique_word]["doc_freq"] += 1
                else:
                    if file not in positional_index[unique_word]["docs"]:
                        positional_index[unique_word]["docs"][file] = [word_pos]
                    else:
                        positional_index[unique_word]["docs"][file].append(word_pos)                        

print("total terms :", len(positional_index))

total terms : 8970


#### Pickle the postitional index

In [26]:
# pickle the positional index
path = "G:\\IIITD\\Sem 2\\IR\\assignment\\A1"
os.chdir(path)


with open("positional_index.pickle", "wb") as f:
    pickle.dump(positional_index, f)
    print("Positional index stored successfully")

Positional index stored successfully


In [27]:
# load the bigram inverted index
with open("positional_index.pickle", "rb") as f:
    positional_index = pickle.load(f)
    print("Positional index loaded successfully")

Positional index loaded successfully


In [28]:
positional_index

{'lift': {'doc_freq': 1,
  'docs': {'cranfield0001': [15, 48, 58, 62],
   'cranfield0069': [46],
   'cranfield0086': [51, 58, 60],
   'cranfield0141': [30],
   'cranfield0146': [90, 97],
   'cranfield0147': [10, 18],
   'cranfield0163': [127, 148],
   'cranfield0164': [41],
   'cranfield0200': [11],
   'cranfield0203': [23, 28],
   'cranfield0204': [64],
   'cranfield0206': [87, 101, 133, 144],
   'cranfield0225': [98, 133, 157],
   'cranfield0226': [7, 27],
   'cranfield0229': [101],
   'cranfield0230': [85, 97],
   'cranfield0234': [9, 18],
   'cranfield0235': [70],
   'cranfield0245': [30, 44, 59],
   'cranfield0246': [73],
   'cranfield0247': [9],
   'cranfield0250': [3],
   'cranfield0252': [48],
   'cranfield0253': [79],
   'cranfield0275': [1, 24],
   'cranfield0279': [26, 49],
   'cranfield0288': [41],
   'cranfield0311': [73],
   'cranfield0312': [23],
   'cranfield0360': [0, 8, 21, 29, 43],
   'cranfield0363': [11, 27, 60],
   'cranfield0379': [65],
   'cranfield0416': [17],


## Phrase Queries

### For bigram inverted index

In [30]:
# function to preprocessing input queries
def query_preprocessing(text):
    text_lower = text.lower()
    tokens = word_tokenize(text_lower)
    
    stop_words = set(stopwords.words('english'))
    stopword_removed = [token for token in tokens if token not in stop_words]
    preprocessed_text = [token for token in stopword_removed if token.isalnum()]
    return " ".join(preprocessed_text)

In [31]:
def document_intersection(d1, d2):
    p, q = 0, 0
    answer = []
    
    while p < len(d1) and q < len(d2):
        if d1[p] == d2[q]:
            answer.append(d1[p])
            p += 1
            q += 1
        elif d1[p] < d2[q]:
            p += 1
        else:
            q += 1
    return answer

In [32]:
# function to retrieve query from bigram invered index
def bigram_inverted_index_retrieval(query_num, query):
    tokens = query.split()
    retrival = []
    flag = True

    if len(tokens) > 1:
        for i in range(len(tokens)-1):
            # create bigrams
            bigram = tokens[i] + " " + tokens[i+1]

            if bigram in bigram_inv_index:
                retrival.append((bigram_inv_index[bigram]["doc_freq"], bigram_inv_index[bigram]["docs"]))
            else:
                print("Number of documents retrieved for query {} using bigram inverted index: {}".format(query_num, 0))
                print("Names of documents retrieved for query {} using bigram inverted index: {}".format(query_num, None))
                return

        retrival.sort(key=lambda x: x[0])
        matched_docs = retrival[0][1]
        for i in range(1, len(retrival)):
            matched_docs = document_intersection(matched_docs, retrival[i][1])
        num_matched_docs = len(matched_docs)
        matched_docs_names = ", ".join(matched_docs)
        print("Number of documents retrieved for query {} using bigram inverted index: {}".format(query_num, num_matched_docs))
        print("Names of documents retrieved for query {} using bigram inverted index: {}".format(query_num, matched_docs_names))    
    else:
        print("Number of documents retrieved for query {} using bigram inverted index: 0".format(query_num, 0))
        print("Names of documents retrieved for query {} using bigram inverted index: none".format(query_num))

###  For positional index

In [33]:
# function to retirieve query from positional index
def positional_index_retrieval(query_num, query):
    tokens = query.split()
    retrieval = []
    
    if len(tokens) > 0: 
        for i in range(len(tokens)):
            term = tokens[i]
            if term in positional_index:
                retrieval.append((positional_index[term]["doc_freq"], positional_index[term]["docs"]))
            else:
                print("Number of documents retrieved for query {} using positional index: {}".format(query_num, 0))
                print("Names of documents retrieved for query {} using positional index: {}".format(query_num, None))
                return    
        
        matched_docs = list(retrieval[0][1].keys())
        for i in range(1, len(retrieval)):
            matched_docs = document_intersection(matched_docs, list(retrieval[i][1].keys()))
            
        result = []
        if len(matched_docs) > 0:
            for doc in matched_docs:
                dummy = []
                token1_pos = positional_index[tokens[0]]["docs"][doc]
                for idx in range(1, len(tokens)):
                    temp = []
                    token2_pos = positional_index[tokens[idx]]["docs"][doc]
                    for x in token1_pos:
                        for y in token2_pos:
                            if (x + 1) == y:
                                temp.append(y)
                    if len(temp) == 0:
                        dummy = []
                        break
                    else:
                        token1_pos = temp
                        dummy = temp
                if len(dummy) != 0:
                    result.append(doc)               
            matched_docs_names = ", ".join(result) if result else None
            print("Number of documents retrieved for query {} using positional index: {}".format(query_num, len(result)))
            print("Names of documents retrieved for query {} using positional index: {}".format(query_num, matched_docs_names))
# 
    else:
        print("Number of documents retrieved for query {} using positional index: 0".format(query_num, 0))
        print("Names of documents retrieved for query {} using positional index: none".format(query_num))    

### Evaluate User input query

In [37]:
# take user input
N = int(input("Enter number of queries to excecute : "))
query_list = []
x = 0

while x < N:
    query = input(f"Enter Query {x+1} : ")
    preprocessed_query = query_preprocessing(query)
    query_list.append(preprocessed_query)
    x += 1


# retrieval from index
print()
for i in range(len(query_list)):
    # retrieve from bigram inverted index
    bigram_inverted_index_retrieval(i+1, query_list[i])

    # retrieve from positional index
    positional_index_retrieval(i+1, query_list[i])
    print()

    
    
# wave boundary layer
# curved shock wave
# slipstream experimental investigation

Enter number of queries to excecute : 3
Enter Query 1 : curved shock wave
Enter Query 2 : wave boundary layer
Enter Query 3 : slipstream experimental investigation

Number of documents retrieved for query 1 using bigram inverted index: 2
Names of documents retrieved for query 1 using bigram inverted index: cranfield0002, cranfield0401
Number of documents retrieved for query 1 using positional index: 1
Names of documents retrieved for query 1 using positional index: cranfield0002

Number of documents retrieved for query 2 using bigram inverted index: 7
Names of documents retrieved for query 2 using bigram inverted index: cranfield0002, cranfield0170, cranfield0256, cranfield0308, cranfield0309, cranfield0569, cranfield1157
Number of documents retrieved for query 2 using positional index: 7
Names of documents retrieved for query 2 using positional index: cranfield0002, cranfield0170, cranfield0256, cranfield0308, cranfield0309, cranfield0569, cranfield1157

Number of documents retrieved 